<a href="https://colab.research.google.com/github/akhilpranjal/House-Prices---Advanced-Regression-Techniques/blob/main/My_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries:


In [ ]:
!pip install tensorflow_decision_forests -U
!pip install ydf -U
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib as plt
import numpy as np
import tensorflow_decision_forests as tfdf
import ydf

Load the dataset:

In [ ]:
dataset_df = pd.read_csv(r"/content/train.csv")
print("Full train dataset shape is {}".format(dataset_df.shape))

Full train dataset shape is (1460, 81)


In [ ]:
dataset_df.head(3)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500


In [ ]:
dataset_df=dataset_df.drop("Id", axis=1)

In [ ]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   Alley          91 non-null     object 
 6   LotShape       1460 non-null   object 
 7   LandContour    1460 non-null   object 
 8   Utilities      1460 non-null   object 
 9   LotConfig      1460 non-null   object 
 10  LandSlope      1460 non-null   object 
 11  Neighborhood   1460 non-null   object 
 12  Condition1     1460 non-null   object 
 13  Condition2     1460 non-null   object 
 14  BldgType       1460 non-null   object 
 15  HouseStyle     1460 non-null   object 
 16  OverallQual    1460 non-null   int64  
 17  OverallCond    1460 non-null   int64  
 18  YearBuil

In [ ]:
dataset_df['SalePrice'].describe()

,SalePrice
count,1460.000000
mean,180921.195890
std,79442.502883
min,34900.000000
25%,129975.000000
50%,163000.000000
75%,214000.000000
max,755000.000000


In [ ]:
list(set(dataset_df.dtypes.tolist()))

[dtype('float64'), dtype('O'), dtype('int64')]

Selecting data with only numerical values in it:

In [ ]:
df_num = dataset_df.select_dtypes(include = ['float64', 'int64'])
df_num.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
0,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,61,0,0,0,0,0,2,2008,208500
1,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,298,0,0,0,0,0,0,5,2007,181500
2,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,42,0,0,0,0,0,9,2008,223500
3,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,35,272,0,0,0,0,2,2006,140000
4,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,192,84,0,0,0,0,0,12,2008,250000


Splitting Dataset into training data and testing data:

In [ ]:
def split_dataset(dataset, split_ratio):
    test_index = np.random.rand(len(dataset))<split_ratio
    return dataset[~test_index], dataset[test_index]

train_data, test_data = split_dataset(df_num, 0.3)
print(f"Train data = {train_data.shape} & Test data = {test_data.shape}")

Train data = (1027, 37) & Test data = (433, 37)


In [ ]:
model = ydf.RandomForestLearner(label="SalePrice", task=ydf.Task.REGRESSION, num_trees=100).train(train_data)

print(model.describe())

Train model on 1027 examples
Model trained in 0:00:00.731718


In [ ]:
test_data.shape

(433, 37)

In [ ]:
evaluation = model.evaluate(test_data)
print(evaluation)

RMSE: 29453.9
num examples: 433
num examples (weighted): 433



In [ ]:
testing = pd.read_csv(r"/content/test.csv")

ids = testing.pop('Id')

preds = model.predict(testing)

output = pd.DataFrame({'Id': ids, 'SalePrice': preds})

output.to_csv("submission.csv", index=False)
